In [8]:
# Advanced deep-learning best practices: Introduction to the functional API

from tensorflow.keras import Input, layers

import warnings
warnings.filterwarnings('ignore')

from pandas.plotting import scatter_matrix

# Preparing higher-resolution data generator for the jena datset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential

from tensorflow.keras import layers 

from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import applications, layers, Input


%matplotlib inline
import numpy as np
import datetime
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.datasets import make_blobs
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn import utils

from pylab import mpl, plt
import datetime
import tensorflow as tf
from tensorflow import keras
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

In [9]:
# Reweighting a probality distribution to a different temprature 

import numpy as np 

# orginal distribution is a 1D np array of prebability values that must sum to 1.
# temperature is a factor quantifying the entropy is a factor quantifying the entropy of the output distribution.
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution)/ temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)

# Returns a reweighted version of the orginal distrbution. The sum of the distribution may no longer be 1,
    # so divide it by its sum ot ontrain the new distribution.

In [16]:
# Implementing character-level LSTM text generation

path = tf.keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()

print('Corpus length:', len(text))

Corpus length: 600901


In [28]:
# Vectorizing sequences of charecters 

maxlen = 60 

step = 3 

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of seqiences:', len(sentences))

chars = sorted(list(set(text)))

print('Unique characters:', len(chars))

char_indeces = dict((char, chars.index(char)) for char in chars)

print("Vectorization...")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i , sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indeces[char]] =1
    y[i, char_indeces[next_chars[i]]] = 1

Number of seqiences: 200281
Unique characters: 59
Vectorization...


In [38]:
"""
Building The NetWork:
This network is a single LSTM layer followed by a Dense classifier and softmax over all possible charecters.
Note: 1D convnets also have proven extremly successful at this task in rencent times.
"""

import tensorflow

from tensorflow.keras import models


char_preModel = tensorflow.keras.models.Sequential()

char_preModel.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))

char_preModel.add(layers.Dense(len(chars), activation='softmax'))

# Used categorical_crossentropy to deal with one-hot encoded targets.

# Model compilation configuration 

optimizer = keras.optimizers.RMSprop(lr=0.01)

char_preModel.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [40]:
# Function to sample the next character given the model's predictions 

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [41]:
"""
Finally, the following loop repeatedly trains and generates text.
You begin generating text using a range of different temeparatures after every epoch. 
This allows you to see how the genarated text evolves as the model begins to convege,
as well as the impact of temperature in the sampling strategy.
"""


'\nFinally, the following loop repeatedly trains and generates text.\nYou begin generating text using a range of different temeparatures after every epoch. \nThis allows you to see how the genarated text evolves as the model begins to convege,\nas well as the impact of temperature in the sampling strategy.\n'

In [ ]:
# Text-generation loop

import random 
import sys

for epoch in range(1, 60):
    print('epoch',epoch)
    char_preModel.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    

In [66]:
# Text-generation loop

import random 
import sys

for epoch in range(1, 60):
    print('epoch',epoch)
    char_preModel.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indeces[char]] = 1 
        
        preds = char_preModel.predict(sampled, verbose=0)[:0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
    
        generated_text += next_char
        generated_text = generated_text[1:]
    
        sys.stdout.write(next_char)

epoch 1
Train on 200281 samples
 28288/200281 [===>..........................] - ETA: 1:24 - loss: nan

KeyboardInterrupt: 

In [67]:
# DeepDream:
import tensorflow

from tensorflow.keras.applications import inception_v3

#from tensorflow.keras import backed as K

#K.set_learning_phase(0)

deepMindModel = inception_v3.InceptionV3(weights='imagenet',
                                        include_top=False)


AttributeError: 'str' object has no attribute 'decode'

In [ ]:
# Setting up the DeepDream configuration 

layer_contributions = {   # Dictionary mapping layers names to a coefficient quantifying how much the layers ativation
    'mixed2': 0.2,        # cantributes to the loss you'll seek to maxize.
    'mixed3': 3.,         # NOTE The layer names are hardcored in hte built-in inception V3 application.
    'mixed4': 2.,         # List all layer names using model.summary()
    'mized5': 1.5,
}

In [71]:
'''
Now define a tensor that contains the loss:
The weighted sum of the L2 norm of the activation of the layers in listing above.


# Defining the loss to be maximized 

'''

#layer_dict = dict([(layer_contributions.name, layer) for layer in model.layers])

loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2, :])) / scaling

NameError: name 'K' is not defined

In [118]:
s = '1 w 2 r 3g'


def solve(s):
    s = s.title()[:-1]
    return s +'g'

solve(s)    

'1 W 2 R 3g'

In [121]:

# Gradient-ascent process

dream = deepMindModel.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(k.mean(k.abs(grads)), le-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_andgrads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    
    return loss_value, grad_values


def gradient_ascent(x, inrations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_andgrads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        
        x += step * grad_values
        
    return x



NameError: name 'deepMindModel' is not defined

In [128]:
# Running gradient ascent over different successive scales

import numpy as np 

step = 0.1 
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10

base_image_path = '....'


img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]

successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
                  for dim in orginal_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

orginal_img = np.copy(img)

shrunk_original_img = resize_img(img, successive_shapes[0])


for shape in successive_shapes:
    print('Processing image shape', shape)
    
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                         iterations=interations,
                         step=step,
                         max_loss=max_loss)
    
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(orginal_img, shape)
    lost_details = same_size_original - upscaled_shrunk_original_img
    
    img += lost_details
    
    shrunk_original_img = resize_img(original_shape, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
    
saave_img(img, fname='final_dream.png')

NameError: name 'preprocess_image' is not defined

In [131]:
# Auxlliary functions 

import scipy

from tensorflow.keras.preprocessing import image


def resize_img(img, size):
    
    img = np.copy(img)
    
    factors = (1,
              float(size[0]) / img.shape[1],
              float(size[1]) / img.shape[2],
              1)
    
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    
    img = np.expand_dims(img, axis=0)
    
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        
        x /= 2.
        x += 0.5 
        x *= 255.
        x = np.clip(x, 0, 255).astype('uint8')
        
        return x 

In [134]:
"""
NOTE: Because the original INception V3 network was trained to recognize concepts in images of size 299 x 299,
and given that the orecess inclves scaling the images that aare somewhere between 300 x 300 and 400 x 400.
Regarless, you can run the same code on images of any size and ratio


# Neural style transfer 

loss = distance(style(reference_image) - style(generated_image)) + distance(content(orginal_image) - content(generated_image))

"""



'\nNOTE: Because the original INception V3 network was trained to recognize concepts in images of size 299 x 299,\nand given that the orecess inclves scaling the images that aare somewhere between 300 x 300 and 400 x 400.\nRegarless, you can run the same code on images of any size and ratio\n\n\n# Neural style transfer \n\nloss = distance(style(reference_image) - style(generated_image)) + distance(content(orginal_image) - content(generated_image))\n\n'

In [135]:
"""
Neural style transfer in Keras:


Defining the paths to the style-reference iamge and the target image.

"""

from tensorflow.keras.preprocessing.image import load_img, img_to_array

target_image_path  = 'img/portrait.jpg'

style_referenve_image_path = 'img/transfer_style_reference.jpg'

width, height = load_img(target_image_path).size

img_height = 400

img_width = int(width * img_height / height)


FileNotFoundError: [Errno 2] No such file or directory: 'img/portrait.jpg'

In [136]:
# Auxillary fucntions 

from tensorflow.keras.applications import vgg19

In [141]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    
    img = img_to_array(img)
    
    img = np.expand_dims(img, axis=0)
    
    img - vgg19.preprocess_input(img)
    
    return img



def deprecess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] +=123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [149]:
# Loading the pretraing VGG19 network fand applying it ti the three images 

from tensorflow.keras import backend as K


target_image = K.constant(preprocess_image(target_image_path))

style_reference_image = K.constant(preprocess_image(style_referenve_image_path))

combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image,
                             style_reference_image,
                             combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                   weights='imagenet',
                   include_top=False)

print('Model loaded.')

def contect_loss(base, combination):
    return K.sum(K.square(combination - base))


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    
    channels = 3
    
    size = img_height * img_width
    
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

NameError: name 'img_height' is not defined

In [150]:
# Total variation loss 

def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    
    b = K.square(x[:, :img_height - 1, :img_width -1, :] - x[:, :img_height - 1, 1:, :])
    
    return K.sum(K.pow(a + b, 1.25))

In [153]:
# Definig the final loss that you'll minize 

out_puts_dict = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

total_variation_weight = 1e-4

style_weight = 1.

content_weight = 0.025

loss = K.variable(0.)

layer_features = outputs_dict[content_layer]

target_image_features = layer_features[0, :, :, :]

combination_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(targer_image_features,
                                     combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[content_layer]
    style_reference_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    
loss += total_variation_weight * total_variation_loss(combination_image)

NameError: name 'model' is not defined

In [157]:
# Setting up the gradient-descent process 

loss = K.variable(0.)

grads  = K.gradients(loss, combination_image)[0]


fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grad_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
    
        return grad_values
    
evaluator = Evaluator()

NameError: name 'combination_image' is not defined

In [161]:
#from scipy.optimize import fmin_l_bfgs_b

#from scipy.misc import imsave

import time 

In [ ]:
# Style-transfer loop 

result_prefix = 'my_result'

x = preprocess_image(target_image_path)

x = x.flatten()

for i in range(iterations):
    print('Start of iteration', i)
    
    start_time = time.time()
    
    x, min_val, info = fmin_1_bfgs_b(evaluator.grads,
                                    maxfun=20)
    
    print('Current loss value:', min_val)
    img = x.copy().resahape((img_height, img_width, 3))
    
    img = deprocess_image(img)
    
    fname = result_prefix + '_at_iteratio_%d.png' % i
    
    imsave(fname, img)
    
    print('Image saved as'. fname)
    
    end_time = time.time()
    
    print('Iteration %d completed in %ds' % (i, end_time - start_time))
    

In [ ]:
# Variational autoencoders 

In [162]:
z_mean, z_log_variencee = encoder(input_img) # encodes the input into a mean and variance parameter 

z = z_mean + exp(z_log_variance) * epsilon # Draws a latent point using a small random epsilon 

reconstructed_img = decoder(z) 

model = Model(input_img, reconstructed_img) # Instantiates the autoencoder model,
                                            # which maps an input image ot its reconstruction

NameError: name 'encoder' is not defined

In [ ]:
# VAE encoder network 

import tensorflow.keras 

from tensorflow.keras import layers

from tensorflow.keras.models import Model 

import numpy as np 


img_shape = (28, 28, 1)

batch_size = 16

latent_dim = 2 

input_img = keras.Input(shape=img_shape)

In [165]:
x = layers.Conv2D(32, 3,
                 padding='same', activation='relu')(input_img)

x = layers.Conv2D(64, 3,
                 padding='same', activation='relu', strides=(2, 2))(x)

x = layers.Conv2D(64, 3,
                 padding='same', activation='relu')(x)

x = layers.Conv2D(64, 3,
                 padding='same', activation='relu')(x)

shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)

x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)

z_log_var = layers.Dense(latent_dim)(x)

# latent-space_sampling function


def sampling(args):
    z_mean, z_log_var = args 
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                             mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])



In [168]:
# VAE decoder network, mapping latent spce points to images

decoder_input = layers.Input(K.int_shape(z)[1:])

x = layers.Dense(np.prod(shape_before_flattening[1:]),
                activation='relu')(decoder_input)

x = layers.Reshape(shape_before_flattening[1:])(x)

x = layers.Conv2DTranspose(32, 3,
                          padding='same',
                          activation='relu',
                          strides=(2, 2))(x)

x = layers.Conv2D(1, 3, 
                 padding='same',
                 activation='sigmoid')(x)

decoder = Model(decoder_input, x)

z_decoded = decoder(z)


In [170]:
# Custom layer used to compute the VAE loss

class CustomVatiationalLayer(tensorflow.keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        
        x = K.flatten(x)
        
        z_decoded = K.flatten(z_decoded)
        
        xent_loss = tensorflow.keras.metrics.binary_crossentropy(x, z_decoded)
        
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        
        return K.mean(xent_loss + kl_loss)
    
    
    def call(self, inputs):
        x = inputs[0]
        
        z_decoded = inputs[1]
        
        loss = self.vae_loss(x, z_decoded)
        
        self.add_loss(loss, inputs=inputs)
        
        return x
    
y = CustomVatiationalLayer()([input_img, z_decoded])


In [175]:
# Training the VAE

from tensorflow.keras.datasets import mnist

vae = Model(input_img, y)

vae.compile(optimizer='rmsprop', loss=None)

vae.summary()



AttributeError: 'numpy.ndarray' object has no attribute 'op'

In [176]:
(x_train, _), (x_test, y_test) = mnist.load_data()

In [177]:
x_train = x_train.astype('float') / 255
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255
x_test = x_test.reshape(x_test.shape + (1,))


vae.fit(x=x_train, y=None,
       shuffle=True,
       epochs=10,
       batch_size=batch_size,
       validation_data=(x_test, None))

NameError: name 'vae' is not defined

In [182]:
# Sampling a grid of pints from the 2d latent space and decoding them to images


import matplotlib.pyplot as plt 

from scipy.stats import sorm 

n = 15

digit_size = 20

figure = np.zeros((digit_size * n, digit_size * n))

grid_x = norm.ppf(np.linspace(0.05, 0.95, n))

grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi,yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = decoder.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit


        plt.figure(figsize=(10, 10))
        
        plt.imshow(figure, cmap='Greys_r')
        plt.show()

ImportError: cannot import name 'sorm' from 'scipy.stats' (C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\scipy\stats\__init__.py)

In [188]:
import tensorflow.keras 
from tensorflow.keras import layers
import numpy as np

latent_dim = 32 

height = 32 

width = 32 

channels = 3

In [189]:
# The generator: GAN generator network

gene_in = tensorflow.keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(gene_in)

x = layers.LeakyReLU()(x)

x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)

x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)

x = layers.LeakyReLU()(x)

x = layers.Conv2D(256,5, padding='same')(x)

x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)

x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = tensorflow.keras.models.Model(gene_in, x)

generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense_11 (Dense)             (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_201 (Conv2D)          (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 32, 32, 256)       1048

In [193]:
# The discriminator: The GAN discriminator network 

discriminator_in  = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_in)

x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)

x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)

x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x) # One dropout an important trick

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = tensorflow.keras.models.Model(discriminator_in, x)

discriminator.summary()

discriminator_optimizer = tensorflow.keras.optimizers.RMSprop(
    lr = 0.0008,
    clipvalue=1.0,
    decay=1e-8)


discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_211 (Conv2D)          (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_212 (Conv2D)          (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_213 (Conv2D)          (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 6, 6, 128)         0   

In [194]:
# Adversarial network

discriminator.trainableq = False

gan_input = tensorflow.keras.Input(shape=(latent_dim,))

gan_output = discriminator(generator(gan_input))

gan = tensorflow.keras.models.Model(gan_input, gan_output)

gan_optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')



In [196]:
# Implemetning GAN training 

import os 

from tensorflow.keras.preprocessing import image 

(x_train, y_train), (_, _) = tensorflow.keras.datasets.cifar10.load_data()


170500096/170498071 [==============================] - 20s 0us/step


In [202]:
x_train  = x_train[y_train.flatten() == 6]

x_train = x_train.reshape(
    (x_train.shape[0], ) + 
    (height, width, channels)).astype('float32') / 255


iteratios = 10000

batch_size = 20 

# save_dir = '' Specifies where you want to save genarated images

start = 0 

for step in range(iterations):
    random_laten_vectors = np.random.normal(size=(batch_size,
                                                 latent_dim))
    
    generated_images = generator.predict(random_laten_vectors)

    stop = start + batch_size
    real_images = x_train[start: stop]
    
    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)),
                            np.zeros((batch_size, 1))])
    
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size,
                                                  latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))
    
    a_loss = gan.train_on_batch(random_laten_vectors, misleading_targets)
    
    
    start += batch_size
    
    if start > len(x_train) - batch_size:
        start = 0
        
    if step % 1000 == 0:
        gan.sample_weights('gan.h5')
        
        print('Discriminator loss:', d_loss)
        
        print('Adversarial loss:', d_loss)
        
        img = image.array_to_img(generated_images[0] * 255, scale=False)
        
        img.save(os.path.join(save_dir,
                             'generated_frog' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255, scale=False)
        
        img.save(os.path.jaoin(save_dir,
                              'real_frog' + str(step) + '.png'))

IndexError: boolean index did not match indexed array along dimension 0; dimension is 5000 but corresponding boolean dimension is 50000

In [206]:
# To perform binary classificaiton
# And stack of layers with a Dense layer witha single unit 
# and a sigmoid activation and use binary_crossentropy as the loss
# Target should be either 0 or 1

from tensorflow.keras import models, layers

model = models.Sequential()

model.add(layers.Dense(32, activation='relu', input_shape= (num_input_features,)))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy')



NameError: name 'num_input_features' is not defined

In [207]:
"""
To perform single-label categorical classification (where each sample has exctly one class,
no more),end your stack of layers with a Dense layer with a number of units equal to the number of classes,
nd a softmax activation. If you targets are one-hot encoded, use categorical_corssentropy as the loss,
if interges use sparse_categorical_crossentropy:
"""

model = models.Sequential()

model.add(layers.Dense(32, activation='relu', input_shape=(num_input_features)))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

NameError: name 'num_input_features' is not defined

In [209]:
"""
To perform multilabel categorical classification (where each sample can have several classes)
    end your stack of layers with a Dense layer with a number of units equal to the number
        of classes and a sigmoid activation, using binary_crossentropy as loss. TARGETS SHOULD BE K HOT ENCODED:
    
"""

model = models.Sequential()

model.add(layers.Dense(32, activation='relu', input_shape=(num_input_features)))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(num_classes, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss= 'binary_crossentropy')



NameError: name 'num_input_features' is not defined

In [210]:
"""
To perform regressions toward a vector of continous values, end your stack of layers with a Dense layer witha number of 

units equal to the number of values youre trying to predict.

(often a single one, such as the price of a house), and no activation.
    Several losses can be used for regression, most commonly mean_squared_error (MSE) and mean_absolute_error (MAE):
"""
model = models.Sequential()

model.add(layers.Dense(32, activation='relu', input_shape=(num_input_features)))

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(num_values))

model.compile(optimizer='rmsprop', loss='mse')


NameError: name 'num_input_features' is not defined

In [217]:
# convnets: Here's a typical image-classifcation network
# (categorical classification)

model = models.Sequential()

model.add(layers.SeparableConv2D(32, 3, activation='relu',
                                input_shape=(height, width, channels)))

model.add(layers.SeparableConv2D(64,3, activation='relu'))
          
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64,3, activation='relu'))
model.add(layers.SeparableConv2D(128,3, activation='relu'))
model.add(layers.MaxPooling2D(2))
          
model.add(layers.SeparableConv2D(64,3, activation='relu'))
model.add(layers.SeparableConv2D(128,3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())
          
model.add(layers.Dense(32, activation='relu'))
          
model.add(layers.Dense(num_classes, activations='softmax'))
          
model.compile(optimizer='remsprop', loss='categorical_crossentropy')
          

          

NameError: name 'num_classes' is not defined

In [219]:
# A single RNN later for binary classification of vactor sequences:

model = models.Sequential()

model.add(layers.LSTM(32, input_shape=(num_timesteps, num_features)))


model.add(layers.Dense(num_classes, activation='sigmoid'))

model.compile(optimizer='rsmprop', loss='binary_crossentropy')



NameError: name 'num_timesteps' is not defined

In [220]:
# This is a stacked RNN layer for niabry classifaction of vector swquences:

model = models.Sequential()

model.add(layers.LSTM(32, return_sequences=True,
                     input_shape=(num_timesteps, num_features)))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(32))

model.compile(optimizer='rmsprop', loss='binary_corssentropy')

NameError: name 'num_timesteps' is not defined